# 3.1 Функции в Python – это объекты первого класса

Функции Python относятся к **объектам первого класса**: их можно присваивать переменным, хранить их в структурах данных, передавать их в качестве аргументов другим функциям и даже возвращать их в качестве значений из других функций.

На протяжении всей этой главы будет использоваться привелённая ниже функция `yell`. Это простой игрушечный пример с легко распознаваемым результатом:

In [1]:
def yell(text: str) -> str:
    return text.upper() + '!'

In [2]:
yell('привет')

'ПРИВЕТ!'

## Функции — это объекты

Поскольку функция `yell` в Python является объектом, вы можете ее присвоить еще одной переменной, точно так же, как это происходит с любымдругим объектом:

In [3]:
bark = yell

Эта строка кода не вызывает функцию. Она берет объект-функцию, на который ссылается имя `yell`, и создает второе имя, `bark`, которое на него указывает. 

In [4]:
bark('гав')

'ГАВ!'

Объекты-функции и их имена — это две отдельные компетенции: вы можете удалить первоначальное имя функции `yell`, и, поскольку еще одно имя `bark` по-прежнему указывает на лежащую в основе функцию, вы все так же можете через него вызвать эту функцию:

In [5]:
del yell

In [6]:
yell('Привет?')

NameError: name 'yell' is not defined

In [ ]:
bark('гав')

In [ ]:
bark.__name__ # just for debug

## Функции могут храниться в структурах данных

In [ ]:
funcs = [bark, str.lower, str.capitalize]

In [ ]:
funcs

In [ ]:
for f in funcs:
    print(f, f('строчка'))

In [ ]:
funcs[0]('приветище')

## Функции могут передаваться другим функциям

Вот функция `greet`, которая форматирует строковое значение приветствия, используя переданный в нее объект-функцию, и затем его печатает:

In [ ]:
def greet(func):
    greeting = func('Привет! Я — программа Python')
    print(greeting)

In [ ]:
greet(bark)

Функции, которые в качестве аргументов могут принимать другие функции, также называются ***функциями более высокого порядка (higher-order functions)***. Они являются непременным условием функционального стиля программирования.

Классическим примером функций более высокого порядка является встроенная функция `map`. Она принимает объект-функцию и итерируемый объект и затем вызывает эту функцию с каждым элементом итерируемого объекта, выдавая результат по мере его прохождения.

In [ ]:
list(map(bark, ['здравствуй', 'эй', 'привет']))

## Функции могут быть вложенными

Python допускает определение функций внутри других функций. Такие функции нередко называются ***вложенными функциями (nested functions)***, или ***внутренними функциями (inner functions)***. Приведем пример:

In [ ]:
def speak(text: str) -> str:
    def whisper(t: str) -> str:
        return t.lower() + '...'
    return whisper(text) 

In [ ]:
speak('Привет, Мир')

функция `whisper` не существует за пределами функции `speak`:

In [ ]:
whisper('Йоу')

Рассмотрим ещё пример. Ниже приведена функция, определяющая две внутренние функции. В зависимости от аргумента, передаваемого в функцию верхнего уровня, она выбирает и возвращает источнику вызова одну из внутренних функций:

In [ ]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper

Это означает, что функции не только могут *принимать линии поведения* через аргументы, но и ***возвращать линии поведения***. 

In [ ]:
speak_func = get_speak_func(0.7)
speak_func('Привет') 

In [ ]:
speak_func = get_speak_func(0.3)
speak_func('Привет') 

## Функции могут захватывать локальное состояние

Мало того что функции могут возвращать другие функции, эти внутренние функции также могут захватывать и уносить с собой часть состояния родительской функции. И что же это означает? Немного перепишем предыдущий пример:

In [ ]:
def get_speak_func(text: str, volume: float):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper

In [ ]:
get_speak_func('Привет, Мир', 0.7)() 

Как видно, теперь `whisper` и `yell` не имеют собственных аргументов, а захватывают значение параметра `text` извне.

Функции, которые это делают, называются ***лексическими замыканиями (lexical closures)*** (или, для краткости, просто *замыканиями*). Замыкание помнит значения из своего лексического контекста, даже когда поток управления программы больше не находится в этом контексте.

Так мы можем предварительно конфигурировать линии поведения. Ниже приведен еще один скелетный пример, который иллюстрирует эту идею:

In [ ]:
def make_adder(n):
    def add(x):
        return x + n
    
    return add

In [ ]:
plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [ ]:
plus_3(4) 

In [ ]:
plus_5(4)

В данном примере `make_adder` служит ***фабрикой*** для создания и конфигурирования *функций-«сумматоров»*. Обратите внимание на то, что *функции-«сумматоры»* по-прежнему могут получать доступ к аргументу `n` функции `make_adder` (объемлющему контексту).

## Объекты могут вести себя как функции

Объекты можно сделать вызываемыми (callable), реализовав для них соответствующий дандер-метод `__call__`:

In [ ]:
class Adder:
    def __init__(self, n):
        self.n = n
        
    def __call__(self, x):
        return self.n + x 

In [ ]:
plus_3 = Adder(3)
plus_3(4)

За кадром «вызов» экземпляра объекта в качестве функции сводится к исполнению метода `__call__` этого объекта.

Безусловно, не все объекты будут вызываемыми. Вот почему существует встроенная функция `callable`, которая проверяет, является объект вызываемым или нет:

In [ ]:
callable(plus_3)

In [ ]:
callable(bark)

In [ ]:
callable(str())

## Ключевые выводы

- В Python абсолютно все является объектом, включая функции. Их можно присваивать переменным, хранить в структурах данных и передавать или возвращать в другие функции и возвращать из них (функции первого класса).
- Функции первого класса позволяют абстрагироваться и раздавать линии поведения в ваших программах.
- Функции могут быть вложенными, и они могут захватывать и уносить с собой часть состояния родительской функции. Функции, которые это делают, называются замыканиями.
- Объекты можно делать вызываемыми. Во многих случаях это позволяет рассматривать их в качестве функций.